In [9]:
import os
import time
import pandas as pd
import openai
import re
import requests
import sys
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity
import tiktoken

openai.api_key = os.getenv("OPENAI_API_KEY") 
openai.organization = os.getenv("OPENAI_ORGANIZATION") 

start_time=time.time()
path ='./QnA/'

########### This helps takes care of removing metadata
search_string = "" 
metadata_counter = 0
############
d = []
text=""

for root, directories, files in os.walk(path , topdown=False):
    for file in files:
        if file.lower().endswith(".txt"):
            name =(os.path.join(root,file))
            f = open(name, "r",encoding="utf-8")
            for line in f:
                text +=line
            f.close()
            d.append({'FILE NAME': file ,'CONTENT': text})
            pd.DataFrame(d)
            metadata_counter = 0
            text=""
end_time = time.time()
duration = end_time - start_time

print ("Script Execution: ", duration)

Script Execution:  0.0027740001678466797


In [11]:
df = pd.DataFrame(d)
df

,FILE NAME,CONTENT
0,"회원가입을 했는데, 서비스 신청을 꼭 해야만 하나요?.txt","회원가입은 파스토 계정을 생성하는 단계이며, [서비스 신청]을 통해 고객님의 정보 ..."
1,고가 상품도 보관 가능한가요?.txt,"고가 상품의 경우, 별도의 공간에서 상품 특성에 맞추어 보관하고 있으며 이 경우 추..."
2,회사소개서를 받아보고 싶어요..txt,아래 링크를 통해 파스토 회사소개서를 다운받으실 수 있습니다.\n 회사소개서 다운받...
3,"사업자등록을 하지 않았는데, 서비스를 이용할 수 있나요?.txt",사업자 등록을 하지 않으신 고객님께서도 일반 회원으로 FASSTO 서비스 이용이 가...
4,입고가 불가한 상품이 있나요?.txt,파스토는 고객님들의 다양한 상품에 대해서 서비스를 제공해드리기 위해 노력하고 있습니...
5,냉동 식품의 경우 얼마간의 판매 예상분을 입고하면 되나요?.txt,물류비 절감을 위해서는 회전율을 고려하여 1~2개월의 판매 예상분을 입고하시는 것이...
6,신용카드는 왜 등록하나요?.txt,"신용카드 등록은 보증금 대용으로 사용되며, 서비스 신청 완료 이후 물류센터로 상품을..."
7,서비스 이용절차는 어떻게 되나요?.txt,[서비스 이용절차]\n 회원가입 → 서비스 신청 → 서비스 심사 및 센터 배정 → ...
8,물류센터 운영시간은 어떻게 되나요?.txt,"파스토 센터의 운영시간은 평일 09시 30분 ~ 17시이며, 상품 입고 가능 시간은..."
9,파스토는 어떤 서비스를 제공하나요?.txt,"파스토는 상품 입고부터 보관, 택배출고, 차량납품, 포장, 스티커부착, 라벨링 등 ..."


In [12]:
# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.replace("#","")
    s = s.strip()
    if s =="":
        s = "<blank>"
    return s
df_normalized=df.copy()
df_normalized['CONTENT'] = df["CONTENT"].apply(lambda x : normalize_text(x))

In [14]:
df_normalized

,FILE NAME,CONTENT
0,"회원가입을 했는데, 서비스 신청을 꼭 해야만 하나요?.txt","회원가입은 파스토 계정을 생성하는 단계이며, [서비스 신청]을 통해 고객님의 정보 ..."
1,고가 상품도 보관 가능한가요?.txt,"고가 상품의 경우, 별도의 공간에서 상품 특성에 맞추어 보관하고 있으며 이 경우 추..."
2,회사소개서를 받아보고 싶어요..txt,아래 링크를 통해 파스토 회사소개서를 다운받으실 수 있습니다. 회사소개서 다운받기 ...
3,"사업자등록을 하지 않았는데, 서비스를 이용할 수 있나요?.txt",사업자 등록을 하지 않으신 고객님께서도 일반 회원으로 FASSTO 서비스 이용이 가...
4,입고가 불가한 상품이 있나요?.txt,파스토는 고객님들의 다양한 상품에 대해서 서비스를 제공해드리기 위해 노력하고 있습니...
5,냉동 식품의 경우 얼마간의 판매 예상분을 입고하면 되나요?.txt,물류비 절감을 위해서는 회전율을 고려하여 1~2개월의 판매 예상분을 입고하시는 것이...
6,신용카드는 왜 등록하나요?.txt,"신용카드 등록은 보증금 대용으로 사용되며, 서비스 신청 완료 이후 물류센터로 상품을..."
7,서비스 이용절차는 어떻게 되나요?.txt,[서비스 이용절차] 회원가입 → 서비스 신청 → 서비스 심사 및 센터 배정 → 신용...
8,물류센터 운영시간은 어떻게 되나요?.txt,"파스토 센터의 운영시간은 평일 09시 30분 ~ 17시이며, 상품 입고 가능 시간은..."
9,파스토는 어떤 서비스를 제공하나요?.txt,"파스토는 상품 입고부터 보관, 택배출고, 차량납품, 포장, 스티커부착, 라벨링 등 ..."


In [15]:
df_blank = df_normalized[df_normalized.CONTENT=='<blank>']
print(df_blank)

Empty DataFrame
Columns: [FILE NAME, CONTENT]
Index: []


In [16]:
df_normalized

,FILE NAME,CONTENT
0,"회원가입을 했는데, 서비스 신청을 꼭 해야만 하나요?.txt","회원가입은 파스토 계정을 생성하는 단계이며, [서비스 신청]을 통해 고객님의 정보 ..."
1,고가 상품도 보관 가능한가요?.txt,"고가 상품의 경우, 별도의 공간에서 상품 특성에 맞추어 보관하고 있으며 이 경우 추..."
2,회사소개서를 받아보고 싶어요..txt,아래 링크를 통해 파스토 회사소개서를 다운받으실 수 있습니다. 회사소개서 다운받기 ...
3,"사업자등록을 하지 않았는데, 서비스를 이용할 수 있나요?.txt",사업자 등록을 하지 않으신 고객님께서도 일반 회원으로 FASSTO 서비스 이용이 가...
4,입고가 불가한 상품이 있나요?.txt,파스토는 고객님들의 다양한 상품에 대해서 서비스를 제공해드리기 위해 노력하고 있습니...
5,냉동 식품의 경우 얼마간의 판매 예상분을 입고하면 되나요?.txt,물류비 절감을 위해서는 회전율을 고려하여 1~2개월의 판매 예상분을 입고하시는 것이...
6,신용카드는 왜 등록하나요?.txt,"신용카드 등록은 보증금 대용으로 사용되며, 서비스 신청 완료 이후 물류센터로 상품을..."
7,서비스 이용절차는 어떻게 되나요?.txt,[서비스 이용절차] 회원가입 → 서비스 신청 → 서비스 심사 및 센터 배정 → 신용...
8,물류센터 운영시간은 어떻게 되나요?.txt,"파스토 센터의 운영시간은 평일 09시 30분 ~ 17시이며, 상품 입고 가능 시간은..."
9,파스토는 어떤 서비스를 제공하나요?.txt,"파스토는 상품 입고부터 보관, 택배출고, 차량납품, 포장, 스티커부착, 라벨링 등 ..."



|MODEL|GENERATION|TOKENIZER|MAX INPUT TOKENS|KNOWLEDGE CUTOFF|
|---|---|---|---|---|
|V2|cl100k_base|8191|Sep 2021|  
|V1|GPT-2/GPT-3|2046|Aug 2020|

In [41]:
tokenizer = tiktoken.get_encoding("cl100k_base")
df_tok=df_normalized.copy()
df_tok['n_tokens'] = df_normalized["CONTENT"].apply(lambda x: len(tokenizer.encode(x)))
df_tok

,FILE NAME,CONTENT,n_tokens
0,"회원가입을 했는데, 서비스 신청을 꼭 해야만 하나요?.txt","회원가입은 파스토 계정을 생성하는 단계이며, [서비스 신청]을 통해 고객님의 정보 ...",122
1,고가 상품도 보관 가능한가요?.txt,"고가 상품의 경우, 별도의 공간에서 상품 특성에 맞추어 보관하고 있으며 이 경우 추...",124
2,회사소개서를 받아보고 싶어요..txt,아래 링크를 통해 파스토 회사소개서를 다운받으실 수 있습니다. 회사소개서 다운받기 ...,92
3,"사업자등록을 하지 않았는데, 서비스를 이용할 수 있나요?.txt",사업자 등록을 하지 않으신 고객님께서도 일반 회원으로 FASSTO 서비스 이용이 가...,104
4,입고가 불가한 상품이 있나요?.txt,파스토는 고객님들의 다양한 상품에 대해서 서비스를 제공해드리기 위해 노력하고 있습니...,1276
5,냉동 식품의 경우 얼마간의 판매 예상분을 입고하면 되나요?.txt,물류비 절감을 위해서는 회전율을 고려하여 1~2개월의 판매 예상분을 입고하시는 것이...,89
6,신용카드는 왜 등록하나요?.txt,"신용카드 등록은 보증금 대용으로 사용되며, 서비스 신청 완료 이후 물류센터로 상품을...",141
7,서비스 이용절차는 어떻게 되나요?.txt,[서비스 이용절차] 회원가입 → 서비스 신청 → 서비스 심사 및 센터 배정 → 신용...,369
8,물류센터 운영시간은 어떻게 되나요?.txt,"파스토 센터의 운영시간은 평일 09시 30분 ~ 17시이며, 상품 입고 가능 시간은...",102
9,파스토는 어떤 서비스를 제공하나요?.txt,"파스토는 상품 입고부터 보관, 택배출고, 차량납품, 포장, 스티커부착, 라벨링 등 ...",239


In [18]:
# Based on https://openai.com/api/pricing/ on 01/29/2023
# If you were using this for approximating pricing with Azure OpenAI adjust the values below with: https://azure.microsoft.com/pricing/details/cognitive-services/openai-service/

#MODEL	USAGE
#Ada     v1	$0.0040 / 1K tokens
#Babbage v1	$0.0050 / 1K tokens
#Curie   v1	$0.0200 / 1K tokens
#Davinci v1	$0.2000 / 1K tokens

#MODEL	USAGE
#Ada     v2	$0.0004 / 1K tokens
#This Ada model, text-embedding-ada-002, is a better and lower cost replacement for our older embedding models. 

n_tokens_sum = df['n_tokens'].sum()

ada_v1_embeddings_cost = (n_tokens_sum/1000) *.0040
babbage_v1_embeddings_cost = (n_tokens_sum/1000) *.0050
curie_v1_embeddings_cost = (n_tokens_sum/1000) *.02
davinci_v1_embeddings_cost = (n_tokens_sum/1000) *.2

ada_v2_embeddings_cost = (n_tokens_sum/1000) *.0004

print("Number of tokens: " + str(n_tokens_sum) + "\n")

print("MODEL        VERSION    COST")
print("-----------------------------------")
print("Ada" + "\t\t" + "v1" + "\t$" + '%.8s' % str(ada_v1_embeddings_cost))
print("Babbage" + "\t\t" + "v1" + "\t$" + '%.8s' % str(babbage_v1_embeddings_cost))
print("Curie" + "\t\t" + "v1" + "\t$" + '%.8s' % str(curie_v1_embeddings_cost))
print("Davinci" + "\t\t" + "v1" + "\t$" + '%.8s' % str(davinci_v1_embeddings_cost))
print("Ada" + "\t\t" + "v2" + "\t$" + '%.8s' %str(ada_v2_embeddings_cost))

Number of tokens: 2675

MODEL        VERSION    COST
-----------------------------------
Ada		v1	$0.0107
Babbage		v1	$0.013375
Curie		v1	$0.0535
Davinci		v1	$0.535
Ada		v2	$0.00107


### Model dimension

https://platform.openai.com/docs/guides/embeddings/what-are-embeddings 

|MODEL|OUTPUT DIMENSIONS|
|---|---|
|Ada|1024|
|Babbage|2048|
|Curie|4096|
|Davinci|12288|


In [26]:
def generate_embeddings(text, model="text-embedding-ada-002"):
    return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']
 
df['ada_v2_embedding'] = df.CONTENT.apply(lambda x: generate_embeddings(x, model='text-embedding-ada-002'))


In [42]:
len(df['ada_v2_embedding'][1])

1536

In [44]:
df_tok = df_tok[df_tok.n_tokens<400]

len(df_tok)

9

### Text search embeddings

- 가장 많이 사용 문서 탐색

|USE CASES|AVAILABLE MODELS|
|---|---|
|Search, context relevance, information retrieval|text-search-ada-doc-001 text-search-ada-query-001 text-search-babbage-doc-001 text-search-babbage-query-001 text-search-curie-doc-001 text-search-curie-query-001 text-search-davinci-doc-001 text-search-davinci-query-001|


### OpenAI Text & Embedding Rate Limits?

- Rate limits are enforced at the organization level, not user level, based on the specific endpoint used as well as the type of account you have.

- Rate limits are measured in two ways: RPM (requests per minute) and TPM (tokens per minute).

- TEXT & EMBEDDING
- Free trial users •20 RPM •150,000 TPM

- Pay-as-you-go users (first 48 hours) •60 RPM •250,000 TPM

- Pay-as-you-go users (after 48 hours) •3,000 RPM •250,000 TPM

- https://beta.openai.com/docs/guides/rate-limits/overview


|TYPE|	1 TPM EQUALS|
|---|---|
|davinci|	1 token per minute|
|curie|	25 tokens per minute|
|babbage|	100 tokens per minute|
|ada|	200 tokens per minute|

In [45]:
import time
from IPython.display import clear_output

request_counter = 0
total_requests_sent = 0
rate_limit= 3000

start_timer=time.time()

def generate_embeddings(text, model="text-embedding-ada-002"):
    global request_counter
    global rate_limit
    global total_requests_sent
    global start_timer
    clear_output(wait=True)
    check_timer = time.time()
    duration = check_timer-start_timer
    print(duration)
    
    if int(duration) >= 60:
        start_timer=time.time()
        request_counter=0
    if request_counter == rate_limit and int(duration) <= 59:
        sleep_for = 60-int(duration)
        print("Sleeping for " + str(sleep_for) +" seconds")
        print("Total requests sent: ", total_requests_sent)
        time.sleep(sleep_for)
        start_timer = time.time()
        request_counter =0
    if request_counter < rate_limit:
        request_counter+=1
        total_requests_sent+=1
        print("Request counter: ", request_counter)
        print("Total requests sent: ", total_requests_sent)
        
    return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

df_embeddings = df_tok.copy()
df_embeddings['ada_v2_embedding'] = df_tok.CONTENT.transform(lambda x: generate_embeddings(x, model='text-embedding-ada-002'))

df_embeddings

3.782973051071167
Request counter:  9
Total requests sent:  9


,FILE NAME,CONTENT,n_tokens,ada_v2_embedding
0,"회원가입을 했는데, 서비스 신청을 꼭 해야만 하나요?.txt","회원가입은 파스토 계정을 생성하는 단계이며, [서비스 신청]을 통해 고객님의 정보 ...",122,"[0.0030568114016205072, -0.003844072110950947,..."
1,고가 상품도 보관 가능한가요?.txt,"고가 상품의 경우, 별도의 공간에서 상품 특성에 맞추어 보관하고 있으며 이 경우 추...",124,"[0.0180361308157444, -0.011002039536833763, 0...."
2,회사소개서를 받아보고 싶어요..txt,아래 링크를 통해 파스토 회사소개서를 다운받으실 수 있습니다. 회사소개서 다운받기 ...,92,"[-0.008481170050799847, 0.00138533441349864, 0..."
3,"사업자등록을 하지 않았는데, 서비스를 이용할 수 있나요?.txt",사업자 등록을 하지 않으신 고객님께서도 일반 회원으로 FASSTO 서비스 이용이 가...,104,"[-0.003308518324047327, -0.02395676262676716, ..."
5,냉동 식품의 경우 얼마간의 판매 예상분을 입고하면 되나요?.txt,물류비 절감을 위해서는 회전율을 고려하여 1~2개월의 판매 예상분을 입고하시는 것이...,89,"[-0.00674519082531333, -0.02381369099020958, 0..."
6,신용카드는 왜 등록하나요?.txt,"신용카드 등록은 보증금 대용으로 사용되며, 서비스 신청 완료 이후 물류센터로 상품을...",141,"[-0.007321336306631565, -0.019554371014237404,..."
7,서비스 이용절차는 어떻게 되나요?.txt,[서비스 이용절차] 회원가입 → 서비스 신청 → 서비스 심사 및 센터 배정 → 신용...,369,"[0.00035062094684690237, 0.0010457402095198631..."
8,물류센터 운영시간은 어떻게 되나요?.txt,"파스토 센터의 운영시간은 평일 09시 30분 ~ 17시이며, 상품 입고 가능 시간은...",102,"[-0.0005161305307410657, 0.001048737671226263,..."
9,파스토는 어떤 서비스를 제공하나요?.txt,"파스토는 상품 입고부터 보관, 택배출고, 차량납품, 포장, 스티커부착, 라벨링 등 ...",239,"[0.0068297921679914, -0.0012520139571279287, 0..."


In [48]:
# search embedded docs based on cosine similarity

df_similarities = df_embeddings.copy()

def get_embedding(text, model="text-embedding-ada-002"):
   return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

def search_docs(df, user_query, top_n=3, to_print=True):
    embedding = get_embedding(
        user_query,
        model="text-embedding-ada-002"
    )

    df_similarities["similarities"] = df_embeddings.ada_v2_embedding.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df_similarities.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res

question = input("무엇을 도와드릴까요?\n\n")

res = search_docs(df, question, top_n=3)

,FILE NAME,CONTENT,n_tokens,ada_v2_embedding,similarities
9,파스토는 어떤 서비스를 제공하나요?.txt,"파스토는 상품 입고부터 보관, 택배출고, 차량납품, 포장, 스티커부착, 라벨링 등 ...",239,"[0.0068297921679914, -0.0012520139571279287, 0...",0.870424
2,회사소개서를 받아보고 싶어요..txt,아래 링크를 통해 파스토 회사소개서를 다운받으실 수 있습니다. 회사소개서 다운받기 ...,92,"[-0.008481170050799847, 0.00138533441349864, 0...",0.864256
0,"회원가입을 했는데, 서비스 신청을 꼭 해야만 하나요?.txt","회원가입은 파스토 계정을 생성하는 단계이며, [서비스 신청]을 통해 고객님의 정보 ...",122,"[0.0030568114016205072, -0.003844072110950947,...",0.854982


similarities 유사도를 비교해 원하는 QA를 찾을 수 있다.

In [62]:
[print(sent + '.\n') for sent in res.CONTENT.values[0].split('.')]

파스토는 상품 입고부터 보관, 택배출고, 차량납품, 포장, 스티커부착, 라벨링 등 온라인 판매자에게 특화된 물류 서비스를 제공합니다.

 특히, 자체 개발한 FMS를 통해 네이버 스마트스토어, 사방넷, NHN커머스(구 고도몰) 등 국내 주요 온라인 서비스들과 주문 연동이 가능한 IT 물류 플랫폼입니다.

 파스토의 국내 풀필먼트 서비스 보러가기 더 궁금하신 사항은 [1:1 문의]에 문의 글을 남겨주세요.

.



[None, None, None, None]

In [63]:
df_similarities

,FILE NAME,CONTENT,n_tokens,ada_v2_embedding,similarities
0,"회원가입을 했는데, 서비스 신청을 꼭 해야만 하나요?.txt","회원가입은 파스토 계정을 생성하는 단계이며, [서비스 신청]을 통해 고객님의 정보 ...",122,"[0.0030568114016205072, -0.003844072110950947,...",0.854982
1,고가 상품도 보관 가능한가요?.txt,"고가 상품의 경우, 별도의 공간에서 상품 특성에 맞추어 보관하고 있으며 이 경우 추...",124,"[0.0180361308157444, -0.011002039536833763, 0....",0.818307
2,회사소개서를 받아보고 싶어요..txt,아래 링크를 통해 파스토 회사소개서를 다운받으실 수 있습니다. 회사소개서 다운받기 ...,92,"[-0.008481170050799847, 0.00138533441349864, 0...",0.864256
3,"사업자등록을 하지 않았는데, 서비스를 이용할 수 있나요?.txt",사업자 등록을 하지 않으신 고객님께서도 일반 회원으로 FASSTO 서비스 이용이 가...,104,"[-0.003308518324047327, -0.02395676262676716, ...",0.828535
5,냉동 식품의 경우 얼마간의 판매 예상분을 입고하면 되나요?.txt,물류비 절감을 위해서는 회전율을 고려하여 1~2개월의 판매 예상분을 입고하시는 것이...,89,"[-0.00674519082531333, -0.02381369099020958, 0...",0.773964
6,신용카드는 왜 등록하나요?.txt,"신용카드 등록은 보증금 대용으로 사용되며, 서비스 신청 완료 이후 물류센터로 상품을...",141,"[-0.007321336306631565, -0.019554371014237404,...",0.785221
7,서비스 이용절차는 어떻게 되나요?.txt,[서비스 이용절차] 회원가입 → 서비스 신청 → 서비스 심사 및 센터 배정 → 신용...,369,"[0.00035062094684690237, 0.0010457402095198631...",0.817186
8,물류센터 운영시간은 어떻게 되나요?.txt,"파스토 센터의 운영시간은 평일 09시 30분 ~ 17시이며, 상품 입고 가능 시간은...",102,"[-0.0005161305307410657, 0.001048737671226263,...",0.849398
9,파스토는 어떤 서비스를 제공하나요?.txt,"파스토는 상품 입고부터 보관, 택배출고, 차량납품, 포장, 스티커부착, 라벨링 등 ...",239,"[0.0068297921679914, -0.0012520139571279287, 0...",0.870424


In [65]:

def search_docs(df, user_query, top_n=3, to_print=True):
    embedding = get_embedding(
        user_query,
        model="text-embedding-ada-002"
    )
    
    df_question = df_similarities.copy()
    df_question["similarities"] = df_similarities.ada_v2_embedding.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df_question.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    return res

res = search_docs(df, question, top_n=1)

ai_question = input("무엇을 도와드릴까요?\n\n")
context= res.CONTENT.values
completion_model='text-davinci-003'

initial_prompt = "AI 도우미 봇 입니다. 원하는 서비스 질문을 대답해드립니다."

combined_prompt = initial_prompt + str(context) + "Q: " + ai_question
response = openai.Completion.create(model=completion_model, prompt=combined_prompt, max_tokens=1000)
ai_response = response['choices'][0]['text'].replace('\n', '').replace(' .', '.').strip()

print("\n"+ ai_response)


.파스토는 요금 정책을 개별 상품 및 사용 기간에 따른 가격을 책정하고 있습니다.이 가격에는 물류 서비스 이용료, 재고관리 수수료, 입고 및 적시 출고 수수료, 택배 및 차량 운송 수수료, 기본 포장 비용 및 패키지 부가 기능 비용, 라벨링 및 스티커 부착 비용 등이 포함되어 있습니다. 또한 이용 기간과 어떤 매장에 사용할지와 같은 상황에 따라 다른 요금 정책이 적용될 수 있습니다. 가격에 대한 보다 자세한 내용은 홈페이지에서 확인해주세요.


In [66]:
[print(sent + '.\n') for sent in ai_response.split('.')]

.

파스토는 요금 정책을 개별 상품 및 사용 기간에 따른 가격을 책정하고 있습니다.

이 가격에는 물류 서비스 이용료, 재고관리 수수료, 입고 및 적시 출고 수수료, 택배 및 차량 운송 수수료, 기본 포장 비용 및 패키지 부가 기능 비용, 라벨링 및 스티커 부착 비용 등이 포함되어 있습니다.

 또한 이용 기간과 어떤 매장에 사용할지와 같은 상황에 따라 다른 요금 정책이 적용될 수 있습니다.

 가격에 대한 보다 자세한 내용은 홈페이지에서 확인해주세요.

.



[None, None, None, None, None, None]